In [1]:
%matplotlib inline
import time
from notebook_init import *

out_root = Path('out/iter')
makedirs(out_root, exist_ok=True)
torch.autograd.set_grad_enabled(True)
rand = lambda : np.random.randint(np.iinfo(np.int32).max)

In [2]:
use_w = True
dataset = 'ffhq'
inst = get_instrumented_model('StyleGAN2', dataset, 'style', device, inst=inst, use_w=use_w)
model = inst.model
model.truncation = 1.0

Loading ../models/checkpoints/stylegan2/stylegan2_ffhq_1024.pt


In [3]:
seeds_ffhq = [366745668]#, 327039870]#, 1302626592, 1235907362, 1150529896, 1881703227]
# seeds_car = [1839348078, 1150529896]#, 1881703227]
# seeds_cat = [889979887,263281582]
# seeds_ffhq = [rand() for _ in range(50)]
start_direction = 0
n_direction = 10
num_frames = 7
num_steps = 12
layer_mode = 'all'
perturb_intensity = 12

In [4]:
save_img = True

for seed in seeds_ffhq:
    print(seed)
    rng = np.random.RandomState(seed)
    noise, z, z_local_basis, z_sv, noise_basis = get_random_local_basis(model, rng)
    layer_start, layer_end = LayerMode(layer_mode)
    strips = []
    with torch.no_grad():
        for trvs_idx in range(start_direction, start_direction+n_direction):
            #batch_frames = create_strip(inst, 'latent', 'style', [z], 0, orient * local_lat_comp[i], 0, local_lat_std[i], perturb_intensity, 0, 18, num_frames=7)[0]
            batch_frames, z_batch = create_strip_iter(inst, 'latent', 'style', trvs_idx, sigma = perturb_intensity, layer_start = layer_start, layer_end = layer_end, 
                                             random_state = None, noise = noise, compare_basis = True, num_frames=num_frames, num_steps = num_steps*2, only_pos=0, scale = 1, verbose=0)
            strips.append(np.hstack(pad_frames(batch_frames[0])))
    grid = np.vstack(strips)

    if save_img:
        Image.fromarray(np.uint8(grid*255)).save(out_root / f'iter_trvs_{dataset}_{seed}_{start_direction}to{start_direction+n_direction-1}_step{num_steps}_ptb{perturb_intensity}_{layer_mode}.jpg') 
    else:
        plt.figure(figsize=(20,40))
        plt.imshow(grid)
        plt.axis('off')
        plt.show()

366745668
Getting iter trvs took 1.1073486804962158
Getting iter trvs took 1.1689033508300781
Getting iter trvs took 1.222881555557251
Getting iter trvs took 1.1012294292449951
Getting iter trvs took 1.1423900127410889
Getting iter trvs took 1.1574020385742188
Getting iter trvs took 1.1662263870239258
Getting iter trvs took 1.1923329830169678
Getting iter trvs took 1.1449370384216309
Getting iter trvs took 1.0948066711425781
